# Questions

1. Characterize the data and comment about its quality
2. Explore and visualize the data e.g. a histogram of trip distance
3. Find interesting trip statistics grouped by hour
4. The taxi drivers want to know what kind of trip yields better tips. Can you build a model for them and explain the model?
5. Pick one of the options below
    - (Option 1) Find an anomaly in the data and explain your findings.
    - (Option 2) Visualize the data to help understand trip patterns

## 1. Characterize the data and comment about its quality
- After downloading the data create some checks
- Review the data dictionary to see of there are weird things about

## 2. Explore and visualize the data e.g. a histogram of trip distance
- Might combine some of the results from the checks in question 1 with this exploration
- I somewhat group finding interesting stats grouped by hour under this umbrella as well
- Make sure to visualize a histogram of tip distance

## 3. Find interesting trip statistics grouped by hour
- For now I will assume this means the start of the trip. However, some trips can laat longer than an hour
 
## 4. The taxi drivers want to know what kind of trip yields better tips. Can you build a model for them and explain the model?
- This is a very vague request, but maybe after interacting with the data some obvious patterns may show up
- There are some leading questions that suggest they may be interested in "Time of day" as well as the length of the trips
- In the model explanation I would be sure to include that it may be beneficial to come to an agreement on what "better tips" mean.
    - Are we talking about just a higher tip?
    - Should we take into consideration cost of drivers time?
    - Longer trips may earn better tips, but a lower percentage of the entire trip cost.
    - Suggestion: use "tip rate" to define better tips. You can also think of this as Percent of Trip Cost.
        - tip/total cost of trip
            - i.e. {'trip_total_cost': '40.00', 'tip': '8.00'}
            - tip_rate = 8/40 = 0.20
    - We can view all these tips on a distribution and either use explicit classification (informed by project management) or ML to generate insights. A really simple approach would be to split the distribution of tip_rates into quartiles (low, med, high, very high). These types of tip ranges can inform a classification model, maybe KNN, of what future trips might yeild. We have the option of harness testing these models to inform users of which trips yeild better tips through simulation OR we can simply use statistics (A/B tests) on the data provided to inform decision makers of the features that are most likely to impact tips.
    - We can also take a PCA approach where we judge the types of trips based on the features with the most influence on the variance. After we have reduced it down to a few influential features we can simulate future outcomes with a few different methods
        - Monte Carlo
            - bootstrapped 
                - "Here is the current distribution of your data, but when you increase x (i.e trip length) by this much here is your likely outcome based on historic behavior."

## 5. Pick one of the options below
#### Find an anomaly in the data and explain your findings.
#### Visualize the data to help understand trip patterns





## 1. Characterize the data and comment about its quality

### First Let's Check out what the data looks like
I can see right away that the columns for the data is shifted. I took a peep at the raw file and noticed that there were a few extra columns saved in the file. Something like this can happen if files were saved by an excel user.

No worries! We will just request the data again.

### NOTES ABOUT DATA QUALITY
- Columns shifted because of extra commas in the source file
- Data Dictionary can be found here: https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf
- String values are coded.
- This is only trip data, we have no information on users.
- There are no hashed user IDs. This could help us identify patterns of spefic taxi drivers
- ehail_fee is a column full of null values, but it is not in the data dictionary
- I did a quick check of store_and_fwd_flag. It does not have mixed data types which is something I have experienced before


### ORDER OF OPERATIONS FOR DATA QUALITY REVIEW
#### 1. High level checks. Reviewing the shape of the data, looking for missing fields, glancing at data types.
    a. Check for null values
    b. Glance at data types
    
#### 2. Data Quality checks. We may not get to all of these checks. But here are some ideas to review the integrity of this data.
    a. Ensure that all data types match
        - There are no Boolean values in store_and_fwd_flag where we expect 'Y'/'N'
        - The dates are all dates and the clocks are all in UTC or labeled with timezone. In this case we don't have timezone. Should we assume UTC or EST since these are all in New York?
    b. Check that lpep_dropoff_datetime >= lpep_pickup_datetime
        - Note I did a check and found 787 records that had the exact same pickup/dropoff time. Maybe this is ok? Review with domain experts to confirm why these events are happening.
    c. Some more advanced quality checks are cross validating the data with the incremental source data.

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
trip_data = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2016-12.csv').sample(100)
trip_data.head()

VendorID lpep_pickup_datetime  \
1 2016-12-18 05:16:10  2016-12-18 05:20:37                    N   
2 2016-12-31 21:50:36  2016-12-31 21:55:46                    N   
1 2016-12-22 00:24:59  2016-12-22 00:36:41                    N   
2 2016-12-15 13:35:34  2016-12-15 13:40:24                    N   
  2016-12-23 10:28:56  2016-12-23 10:49:35                    N   

                       lpep_dropoff_datetime  store_and_fwd_flag  RatecodeID  \
1 2016-12-18 05:16:10                      1                 189         257   
2 2016-12-31 21:50:36                      1                 179         179   
1 2016-12-22 00:24:59                      1                  49          40   
2 2016-12-15 13:35:34                      1                 225         225   
  2016-12-23 10:28:56                      1                  41         138   

                       PULocationID  DOLocationID  passenger_count  \
1 2016-12-18 05:16:10             1          1.30              6.0   
2 2016-12-31 21:50:36             1          0.85              5.5   
1 2016-12-22 00:24:59             2          3.00             11.5   
2 2016-12-15 13:35:34             2          0.87              5.5   
  2016-12-23 10:28:56             5          7.98             24.5   

                       trip_distance  fare_amount  extra  mta_tax  tip_amount  \
1 2016-12-18 05:16:10            0.5          0.5    0.0     0.00         NaN   
2 2016-12-31 21:50:36            0.5          0.5    0.0     0.00         NaN   
1 2016-12-22 00:24:59            0.5          0.5    1.0     0.00         NaN   
2 2016-12-15 13:35:34            0.0          0.5    0.0     0.00         NaN   
  2016-12-23 10:28:56            0.0          0.5    6.0     5.54         NaN   

                       tolls_amount  ehail_fee  improvement_surcharge  \
1 2016-12-18 05:16:10           0.3       7.30                      2   
2 2016-12-31 21:50:36           0.3       6.80                      2   
1 2016-12-22 00:24:59           0.3      13.80                      1   
2 2016-12-15 13:35:34           0.3       6.30                      2   
  2016-12-23 10:28:56           0.3      36.84                      1   

                       total_amount  payment_type  trip_type  
1 2016-12-18 05:16:10             1           NaN        NaN  
2 2016-12-31 21:50:36             1           NaN        NaN  
1 2016-12-22 00:24:59             1           NaN        NaN  
2 2016-12-15 13:35:34             1           NaN        NaN  
  2016-12-23 10:28:56             1           NaN        NaN

### Requesting the data again, overwriting version where VendorID is indexed

This version is looking much better than the previous one

In [9]:
trip_data = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2016-12.csv', sep=',', index_col=False)
trip_data.head()

/var/folders/jj/05h_zgr13_b5d6zynqpmwrl40000gp/T/ipykernel_56846/2846028683.py:1: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.



,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2016-12-01 00:00:54,2016-12-01 00:06:54,N,1,92,192,1,1.29,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2,1
1,2,2016-12-01 00:52:41,2016-12-01 00:54:51,N,1,92,171,1,0.64,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1,1
2,2,2016-12-01 00:10:39,2016-12-01 00:14:47,N,1,75,238,2,0.89,5.5,0.5,0.5,1.70,0.0,NaN,0.3,8.50,1,1
3,2,2016-12-01 00:12:16,2016-12-01 00:15:31,N,1,166,151,1,0.66,4.5,0.5,0.5,1.74,0.0,NaN,0.3,7.54,1,1
4,2,2016-12-01 00:29:22,2016-12-01 00:39:51,N,1,166,42,1,2.15,9.5,0.5,0.5,2.16,0.0,NaN,0.3,12.96,1,1


# 1a checking for null values
This can impact any ML models we may decide to build in the future. Looks like we have a column with all nulls, but this is not provided in the data dictionary. For now we can exclude this from our exercise

Note that ehail_fee is full of nulls, it is not in the data dictionary

In [10]:
trip_data.isnull().sum()

VendorID                       0
lpep_pickup_datetime           0
lpep_dropoff_datetime          0
store_and_fwd_flag             0
RatecodeID                     0
PULocationID                   0
DOLocationID                   0
passenger_count                0
trip_distance                  0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
ehail_fee                1224158
improvement_surcharge          0
total_amount                   0
payment_type                   0
trip_type                      0
dtype: int64

In [11]:
trip_data.dtypes

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                  int64
dtype: object

In [12]:
def check_1(df):
    """
    Quick Example data quality check
    """
    result = df['store_and_fwd_flag'].apply(lambda x: x in ('N','Y')).all()
    
    print(f'Result Pass: {result}')
    assert result
    
check_1(trip_data)

Result Pass: True


# 2b Check that lpep_dropoff_datetime >= lpep_pickup_datetime
- We have trips with exact same start/end times where values seem to reflect a trip is happening
    - Another accuring clock could be valuable in this dataset. We can recommend engineers include an epoch counter
    - Including timezones may help, but probably not applicable for this problem as I don't think the chances of ending a trip exactly at the same time in another timezone realistically occurs

- These trips surfaced a pattern with the data on aggregate where we're getting negative values for total_amount. This is a weird anomaly, did I accidently surface something that can be applied to question 5? Are there refunds?

In [13]:
# here are the trip total_amounts where the dropoff is not (~) after the pickup time. 
trip_data[~(trip_data['lpep_dropoff_datetime'] > trip_data['lpep_pickup_datetime'])]['total_amount']

151         9.80
245        34.56
1945        9.80
1958       27.80
7465        3.00
           ...  
1193880     3.80
1193962     0.80
1196960     1.00
1198510     9.30
1220413     5.80
Name: total_amount, Length: 787, dtype: float64

In [14]:
quick_hist = trip_data[~(trip_data['lpep_dropoff_datetime'] > trip_data['lpep_pickup_datetime'])][['total_amount','VendorID']]

# Showing that there are some total amounts that are less than zero in the entire dataset, not just this subset

In [15]:
(trip_data['total_amount'] < 0).sum()

2800

In [16]:
quick_hist['total_amount_round'] = quick_hist['total_amount'].round()

# What is the distribution of these trips where no time passes?

I would expect these trips to cost some value very close to $0. 

There may be flat rates that are triggered at the end of everytrip, if that is the case, that may explain the high distribution centered around $6.

DISCLAIMER: I aggregated the data before visualizing it. In this case it works because I am ommitting any stats from hover data. But I am aware that this method means that we can't use this figure/ data frame as an object without futher data manipulation

In [17]:
df = quick_hist.groupby(by=['total_amount_round']).count().reset_index()
fig = px.histogram(df, 
                   title='Histogram of Charges per Trip (Where no time passed)',
                   x="total_amount_round", 
                   y="total_amount",
                   labels={'total_amount_round':'Total amount of charge', 'total_amount':'Trip Counts (where no time passed)'},
#                    marginal="box", # or violin, rug
                   nbins=500,
                   hover_data=df.columns)
fig.show()

# Observations
- With only a little bit of time we have already found some interesting characteristics that challange it's quality
- We explored and provided a single visualization of charges where no time has passed in the cab
    - This is related to questions 1, 2, 5

# TO DO:
- Elaborate on question 4, maybe provide some examples or Lucid chart?
- group data by the hour